In [1]:
import boto3
import sagemaker
from sagemaker import Model
import os
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()
print(f"SageMaker Role ARN: {role}")

SageMaker Role ARN: arn:aws:iam::796932308591:role/service-role/SageMaker-ExecutionRole-20250214T145019


In [5]:
from utils import inference_validator, reasoning, feature_engineering, additional_feature_engineering
import pickle, json

In [ ]:
# import boto3
# import pickle
# import json
# import io
# # Initialize S3 client
# s3 = boto3.client('s3')

# # Define S3 bucket and file path
# bucket = 'mitrailabs-personaclassification'

# # Function to load pickle files from S3
# def load_pickle_from_s3(bucket, key):
#     response = s3.get_object(Bucket=bucket, Key=key)
#     return pickle.load(io.BytesIO(response['Body'].read()))

# # Function to load JSON file from S3
# def load_json_from_s3(bucket, key):
#     response = s3.get_object(Bucket=bucket, Key=key)
#     # response_json
#     # io.BytesIO(file_obj['Body'].read())
#     return json.load(response['Body']) 

# # Load all objects from S3
# model_prefix = 'risk_prediction/Intermediate_states/'

# nimputer = load_pickle_from_s3(bucket, f"{model_prefix}nimputer.pkl")
# cimputer = load_pickle_from_s3(bucket, f"{model_prefix}cimputer.pkl")
# label_encoders = load_pickle_from_s3(bucket, f"{model_prefix}label_encoders.pkl")
# inference_preprocessor = load_pickle_from_s3(bucket, f"{model_prefix}inference_preprocessor.pkl")

# before_columns = load_json_from_s3(bucket, f"{model_prefix}before_feature.json")


# model_prefix = 'risk_prediction/models/'

# xgb = load_pickle_from_s3(bucket, f"{model_prefix}xgb_model.pkl")

# # ✅ Now all objects are loaded from S3 into memory
# print("All files loaded successfully from S3!")


In [ ]:
# s3 = boto3.client('s3')

# bucket = 'mitrailabs-personaclassification'


# model_prefix = 'risk_prediction/Intermediate_states/'
# nimputer_response = s3.get_object(
#     Bucket=bucket,
#     Key=f"{model_prefix}nimputer.pkl"
# )

# nimputer = pickle.load(nimputer_response['Body'])


# with open('saved/nimputer.pkl', 'rb') as f:
#     nimputer = pickle.load(f)

# with open('saved/cimputer.pkl', 'rb') as f:
#     cimputer = pickle.load(f)

# with open('saved/label_encoders.pkl', 'rb') as f:
#     label_encoders = pickle.load(f)

    
# with open('saved/inference_preprocessor.pkl', 'rb') as f:
#     inference_preprocessor = pickle.load(f)

# with open('saved/xgb_model.pkl', 'rb') as f:
#     xgb = pickle.load(f)

# with open('saved/before_feature.json', 'r') as f:
#     before_columns = json.load(f)

In [6]:
import boto3
import joblib
import json
import io
import xgboost

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket
bucket = 'mitrailabs-personaclassification'

# Function to load joblib files from S3
def load_joblib_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    return joblib.load(io.BytesIO(response['Body'].read()))

# Function to load JSON file from S3 (unchanged)
def load_json_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    return json.load(response['Body'])

def load_xgboost_from_s3(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    model_data = response['Body'].read()
    
    # Save to a temporary file
    temp_model_path = 'saved/xgb_model.json'
    with open(temp_model_path, 'wb') as f:
        f.write(model_data)
    
    # Load the model
    model = xgboost.Booster()
    model.load_model(temp_model_path)
    return model

# Load all objects from S3
model_prefix = 'risk_prediction/Intermediate_states/'
nimputer = load_joblib_from_s3(bucket, f"{model_prefix}nimputer.joblib")
cimputer = load_joblib_from_s3(bucket, f"{model_prefix}cimputer.joblib")
label_encoders = load_joblib_from_s3(bucket, f"{model_prefix}label_encoders.joblib")
# inference_preprocessor = load_joblib_from_s3(bucket, f"{model_prefix}inference_preprocessor.joblib")
before_columns = load_json_from_s3(bucket, f"{model_prefix}before_feature.json")

model_prefix = 'risk_prediction/models/'
xgb = load_xgboost_from_s3(bucket, f"{model_prefix}xgb_model.json")

# ✅ Now all objects are loaded from S3 into memory
print("All files loaded successfully from S3!")

All files loaded successfully from S3!


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Example user input
user_input = {
    'score_3': 0.5, 'score_4': np.nan, 'score_5': 0.7, 'score_6': np.nan,
    'risk_rate': 0.2, 'last_amount_borrowed': 1000, 'last_borrowed_in_months': 12,
    'credit_limit': 5000, 'income': np.nan, 'ok_since': 24, 'n_bankruptcies': 9,
    'n_defaulted_loans': 1, 'n_accounts': 3, 'n_issues': 2,
    'external_data_provider_credit_checks_last_year': 3,
    'external_data_provider_email_seen_before': 1, 'reported_income': 50000,
    'application_time_in_funnel': 5, 'external_data_provider_credit_checks_last_month': 0,
    'external_data_provider_fraud_score': 50, 'shipping_state': np.nan,
    'facebook_profile': True, 'state': np.nan, 'score_1': np.nan, 'score_2': np.nan, "real_state":'a'
}

# user_input = {
#     'score_3': 510, 'score_4': np.nan, 'score_5': 0.480, 'score_6': 92,
#     'risk_rate': 0.45, 'last_amount_borrowed': 600, 'last_borrowed_in_months': 20,
#     'credit_limit': 500, 'income': 60000, 'ok_since': 32, 'n_bankruptcies': 3.0,
#     'n_defaulted_loans': 0, 'n_accounts': 25.0, 'n_issues': 5.0,
#     'external_data_provider_credit_checks_last_year': 4.0,
#     'external_data_provider_email_seen_before': 5, 'reported_income': 50000,
#     'application_time_in_funnel': 900, 'external_data_provider_credit_checks_last_month': 5.0,
#     'external_data_provider_fraud_score': 250, 'shipping_state': np.nan,
#     'facebook_profile': True, 'state': np.nan, 'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==', 'score_2': np.nan, "real_state": np.nan
# }

user_input = {
    'score_3': 800,  # High credit score
    'score_4': np.nan,  # Missing value
    'score_5': 0.850,  # High probability score
    'score_6': 98,  # High score
    'risk_rate': 0.10,  # Low risk rate
    'last_amount_borrowed': 200,  # Low amount borrowed
    'last_borrowed_in_months': 36,  # Long time since last borrowing
    'credit_limit': 10,  # High credit limit
    'income': 120000,  # High income
    'ok_since': 60,  # Long time since last issue
    'n_bankruptcies': 0.0,  # No bankruptcies
    'n_defaulted_loans': 0,  # No defaulted loans
    'n_accounts': 10.0,  # Moderate number of accounts
    'n_issues': 1.0,  # Few issues
    'external_data_provider_credit_checks_last_year': 1.0,  # Few credit checks
    'external_data_provider_email_seen_before': 10,  # Email seen many times (trusted)
    'reported_income': 120000,  # High reported income
    'application_time_in_funnel': 300,  # Short application time
    'external_data_provider_credit_checks_last_month': 0.0,  # No recent credit checks
    'external_data_provider_fraud_score': 100,  # Low fraud score
    'shipping_state': np.nan,  # Missing value
    'facebook_profile': False,  # Has a Facebook profile
    'state': np.nan,  # Missing value
    'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==',  # Example encrypted score
    'score_2': np.nan,  # Missing value
    'real_state': np.nan  # Missing value
}

user_input = inference_validator(user_input)
user_df = pd.DataFrame([user_input])
# Ensure the user input has the same columns as the training data
user_df = user_df.reindex(columns=before_columns, fill_value=np.nan)

# Assuming you know which features each imputer was trained on
c_features = [f for f in cimputer.feature_names_in_ if f in user_df.columns]
n_features = [f for f in nimputer.feature_names_in_ if f in user_df.columns]
# print(c_features)
# print(n_features)

# Apply only if features exist
if c_features:
    user_df[c_features] = cimputer.transform(user_df[c_features])
if n_features:
    user_df[n_features] = nimputer.transform(user_df[n_features])

user_df = user_df.drop(columns=["target_default"])
# print("target_default" in user_df.columns)
# Apply the same label encoding to the user input
for col in c_features:
    if col in user_df.columns and col != "target_default":
        # Use the transform method of the fitted LabelEncoder
        user_df[col] = label_encoders[col].transform(user_df[col])

yo_features = [i for i in c_features + n_features if i != "target_default"]
# user_df = user_df.reindex(columns=yo_features)
# user_df = user_df.drop(columns=["target_default"])

user_df = feature_engineering(user_df)
user_df = additional_feature_engineering(user_df)
# print(user_df)
# user_processed = inference_preprocessor.transform(user_df)
print(user_df['fraud_score_bin'])
# user_df['fraud_score_bin'] = user_df['fraud_score_bin'].astype(int)
user_df = xgboost.DMatrix(user_df)
# Make prediction
prediction = xgb.predict(user_df)
# prediction_proba = xgb.predict_proba(user_df)

print("Prediction:", prediction)
print(reasoning[str(np.argmax((prediction[0])))])
# print("Prediction Probability:", prediction_proba)
# print(user_processed)
# 6 → 3 → 7 → 1 → 2 → 0 → 5 → 4

0    0
Name: fraud_score_bin, dtype: int64
Prediction: [[0.03845232 0.03844284 0.03844432 0.03844078 0.8462198 ]]
Low-Medium Risk 
 This cluster has a low default rate (0.119) and shows good overall reliability. Borrowers in this cluster have taken moderate-sized loans (last_amount_borrowed: 0.064) and have a low debt-to-income ratio (0.051). Their credit utilization (0.007) is minimal, and their risk scores (score_1, score_2) are moderate. Their behavior suggests slightly more caution is needed compared to low-risk borrowers, but they are generally reliable.


In [ ]:
import sklearn
print(sklearn.__version__)